In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc


In [2]:
fake = faker.Faker("ru_RU")

In [4]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 64
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 128
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [5]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class Data(msgspec.Struct):
    data: Any

In [6]:
index = BPTree[MyKey, Data]()

In [7]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [8]:
data = []
gc.collect()

340

In [9]:
%%time
for i in tqdm(range(10_000_000)):
    val = fake.name() + " - " + str(i)
    if index.insert(MyKey(val)) is None:
        print(f"Duplicate key: {val}")
    data.append(val)

  0%|          | 0/10000000 [00:00<?, ?it/s]

CPU times: user 9min 18s, sys: 8.32 s, total: 9min 26s
Wall time: 9min 23s


In [10]:
index.validate(), gc.collect()

({}, 18)

In [11]:
len(data), len(data) / (60 * 9 + 23)

(10000000, 17761.989342806395)

In [12]:
index.print_node(index.tree[index.root_node])


node-idx:13314, prev:None, next:None, level: 3  <============ *ROOT*
         |-> MyKey(full_name='Валентина Кирилловна Харитонова - 17651'): 132, 37598
         |-> MyKey(full_name='Евдокимов Ульян Витальевич - 1499106'): 37598, 24849
         |-> MyKey(full_name='Козлова Елизавета Станиславовна - 27291'): 24849, 48978
         |-> MyKey(full_name='Матвеева Фёкла Степановна - 159'): 48978, 13313
         |-> MyKey(full_name='Оксана Никифоровна Жданова - 849'): 13313, 54330
         |-> MyKey(full_name='Селиверстов Мирон Владиленович - 359'): 54330, 29050


In [13]:
index.print_node(index.tree[132])


node-idx:132, prev:None, next:37598, level: 2 
      |-> MyKey(full_name='Август Арсенович Миронов - 236371'): 1, 28629
      |-> MyKey(full_name='Авдеева Лукия Артемовна - 1424958'): 28629, 14570
      |-> MyKey(full_name='Аверьян Гертрудович Лобанов - 1464013'): 14570, 28811
      |-> MyKey(full_name='Авксентий Феофанович Кабанов - 6369923'): 28811, 57242
      |-> MyKey(full_name='Агап Гаврилович Лебедев - 271752'): 57242, 7197
      |-> MyKey(full_name='Агата Мироновна Ефимова - 67343'): 7197, 30479
      |-> MyKey(full_name='Агафонов Владлен Гурьевич - 263729'): 30479, 15376
      |-> MyKey(full_name='Агафья Аскольдовна Кулакова - 430222'): 15376, 30222
      |-> MyKey(full_name='Агафья Руслановна Хохлова - 100460'): 30222, 3877
      |-> MyKey(full_name='Адам Филимонович Давыдов - 91164'): 3877, 33121
      |-> MyKey(full_name='Аким Демидович Калашников - 456588'): 33121, 16438
      |-> MyKey(full_name='Аксенова Нонна Эльдаровна - 13223'): 16438, 31845
      |-> MyKey(full_name

In [14]:
index.print_node(index.tree[29050])


node-idx:29050, prev:54330, next:None, level: 2 
      |-> MyKey(full_name='Селиверстов Мирон Владиленович - 359'): 18570, 296
      |-> MyKey(full_name='Семен Антипович Белозеров - 31789'): 296, 47692
      |-> MyKey(full_name='Семенов Сила Брониславович - 21568'): 47692, 24061
      |-> MyKey(full_name='Серафим Изотович Самсонов - 51286'): 24061, 46366
      |-> MyKey(full_name='Сергеева Елена Ниловна - 20594'): 46366, 10956
      |-> MyKey(full_name='Сигизмунд Дмитриевич Зимин - 1100791'): 10956, 39899
      |-> MyKey(full_name='Сидоров Селиверст Бориславович - 1943403'): 39899, 19832
      |-> MyKey(full_name='Сила Гавриилович Лыткин - 42284'): 19832, 40962
      |-> MyKey(full_name='Силин Евсей Ефстафьевич - 8189'): 40962, 5498
      |-> MyKey(full_name='Сильвестр Аксёнович Ефимов - 7165'): 5498, 39874
      |-> MyKey(full_name='Симон Ярославович Дементьев - 19948'): 39874, 21611
      |-> MyKey(full_name='Симонова Ия Яковлевна - 1378090'): 21611, 42841
      |-> MyKey(full_name=

In [16]:
index.min()[1]

MyKey(full_name='Абрамов Август Антипович - 2077679')

In [20]:
index.print_node(index.max()[0])


node-idx:45913, prev:23794, next:None, level: 0 
|-> MyKey(full_name='тов. Якушева Елизавета Валентиновна - 3419865')
|-> MyKey(full_name='тов. Якушева Елизавета Ильинична - 2395298')
|-> MyKey(full_name='тов. Якушева Елизавета Константиновна - 8019064')
|-> MyKey(full_name='тов. Якушева Елизавета Львовна - 4670940')
|-> MyKey(full_name='тов. Якушева Елизавета Романовна - 4418601')
|-> MyKey(full_name='тов. Якушева Елизавета Рудольфовна - 752734')
|-> MyKey(full_name='тов. Якушева Елизавета Филипповна - 884719')
|-> MyKey(full_name='тов. Якушева Жанна Дмитриевна - 4832511')
|-> MyKey(full_name='тов. Якушева Жанна Евгеньевна - 4081811')
|-> MyKey(full_name='тов. Якушева Жанна Святославовна - 4674027')
|-> MyKey(full_name='тов. Якушева Зинаида Архиповна - 7528920')
|-> MyKey(full_name='тов. Якушева Зинаида Вячеславовна - 1758538')
|-> MyKey(full_name='тов. Якушева Зинаида Захаровна - 8525698')
|-> MyKey(full_name='тов. Якушева Зинаида Кирилловна - 1898240')
|-> MyKey(full_name='тов. Яку